In [14]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# import statsmodels.api as sm
# import statsmodels.formula.api as smf


from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from scipy import stats
from scipy.stats import t

In [4]:
df = pd.read_excel("df_scaled.xlsx")
df.head()

,TRANSACTION ID,FECHA NACIMIENTO,FECHA EXP DOC,NIVEL ESTUDIOS,ESTADOCIVIL,PROFESION,GÉNERO,TIPO VIVIENDA,NO PERSONAS A CARGO,ANTIGUEDAD EN LA CIUDAD tiempo en residencia,...,is_outlier_ml,SALDO VENCIDO_LOG,DIAS DE MORA _LOG,TOTAL INGRESOS_LOG,INGRESOS FIJOS_LOG,MONTO APROBADO_LOG,MONTO PREAPROBADO_LOG,NIVEL ESTUDIOS_ENC,ESTADOCIVIL_ENC,GÉNERO_ENC
0,2016918,1983-07-18,2001-07-27 05:00:00,Universitario,Soltero (a),Ingeniero,M,Familiar,-0.292162,-1.001090,...,1,0.0,0.0,1.568333,1.404903,0.871804,0.871804,8,2,1
1,2016914,1965-04-15,1983-08-16 05:00:00,Técnico,Soltero (a),Contador,F,Propia,-0.292162,1.453012,...,1,0.0,0.0,1.849846,1.838373,0.650738,0.650738,7,2,0
2,2016836,2005-01-25,2023-01-31 05:00:00,Técnico,Unión libre,Educador,F,Familiar,-0.292162,0.634978,...,1,0.0,0.0,-1.660744,-1.061758,0.340013,0.340013,7,3,0
3,2016736,1995-11-15,2013-11-28 00:00:00,Técnico,Unión libre,Asesor,F,Propia,-0.292162,-0.455734,...,1,0.0,0.0,0.564594,0.058590,0.495702,0.495702,7,3,0
4,2016576,2006-11-20,2024-11-27 00:00:00,Universitario,Soltero (a),Ingeniero,F,Propia,-0.292162,1.180334,...,1,0.0,0.0,1.612232,1.642078,0.428230,0.428230,8,2,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34156 entries, 0 to 34155
Data columns (total 66 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   TRANSACTION ID                                34156 non-null  int64         
 1   FECHA NACIMIENTO                              34156 non-null  datetime64[ns]
 2   FECHA EXP DOC                                 34156 non-null  datetime64[ns]
 3   NIVEL ESTUDIOS                                34156 non-null  object        
 4   ESTADOCIVIL                                   34156 non-null  object        
 5   PROFESION                                     34156 non-null  object        
 6   GÉNERO                                        34156 non-null  object        
 7   TIPO VIVIENDA                                 34156 non-null  object        
 8   NO PERSONAS A CARGO                           34156 non-null  floa

In [18]:
selected_vars = [
    'SALDO VENCIDO_LOG', 'SCORE', 'ESTADOCIVIL', 'GÉNERO', 'NIVEL ESTUDIOS',
    'TOTAL INGRESOS_LOG', 'INGRESOS FIJOS_LOG', 'PASIVOS', 'ACTIVOS',
    'SALDO CAPITAL', 'AVAL', 'SEGURO', 'PLAZO', 'EDAD', 'ESTRATO'
]

X = df[selected_vars].copy()
X = pd.get_dummies(X, columns=['ESTADOCIVIL', 'GÉNERO', 'NIVEL ESTUDIOS'], drop_first=True)

# Variable dependiente
y = df['MONTO APROBADO_LOG']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [20]:
# Entrenar modelo
modelo_lin = LinearRegression()
modelo_lin.fit(X_train, y_train)

# Predicciones
y_train_pred = modelo_lin.predict(X_train)
y_test_pred  = modelo_lin.predict(X_test)

In [21]:
# Métricas en train
R2_train = r2_score(y_train, y_train_pred)
MSE_train = mean_squared_error(y_train, y_train_pred)
MAE_train = mean_absolute_error(y_train, y_train_pred)

# Métricas en test
R2_test = r2_score(y_test, y_test_pred)
MSE_test = mean_squared_error(y_test, y_test_pred)
MAE_test = mean_absolute_error(y_test, y_test_pred)

In [22]:
print("=== Evaluación Regresión Múltiple ===")
print(f"Intercepto: {modelo_lin.intercept_:.4f}")
print(f"Número de coeficientes: {len(modelo_lin.coef_)}\n")

print(f"Train - R^2: {R2_train:.4f}")
print(f"Train - MSE: {MSE_train:.4f}")
print(f"Train - MAE: {MAE_train:.4f}\n")

print(f"Test  - R^2: {R2_test:.4f}")
print(f"Test  - MSE: {MSE_test:.4f}")
print(f"Test  - MAE: {MAE_test:.4f}")

=== Evaluación Regresión Múltiple ===
Intercepto: 0.3859
Número de coeficientes: 26

Train - R^2: 0.7973
Train - MSE: 0.0050
Train - MAE: 0.0501

Test  - R^2: 0.8068
Test  - MSE: 0.0047
Test  - MAE: 0.0498


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)